In [26]:
# 🔄 Purpose: Fetch latest weekly data for the 5 keywords from Google Trends (weekly granularity only)

from pytrends.request import TrendReq
import pandas as pd
from datetime import date

In [27]:
# Define keywords and timeframe
keywords = ["meditation", "mindfulness", "breathwork", "guided meditation", "yoga nidra"]
today = date.today().strftime("%Y-%m-%d")
timeframe = "today 5-y"

In [28]:
# Initialize pytrends and build payload
pytrends = TrendReq(hl="en-US", tz=360)
pytrends.build_payload(keywords, cat=0, timeframe=timeframe, geo="", gprop="")

In [29]:
# Fetch interest over time
df_weekly = pytrends.interest_over_time()

/Users/saayedalam/jupyterlab-env/lib/python3.13/site-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


In [30]:
# Drop 'isPartial' column if it exists
if 'isPartial' in df_weekly.columns:
    df_weekly = df_weekly[~df_weekly['isPartial']]
    df_weekly.drop(columns='isPartial', inplace=True)

In [31]:
# Reshape to long format
df_weekly = df_weekly.reset_index().melt(id_vars=["date"], var_name="keyword", value_name="search_interest")

In [32]:
# Filter for just the most recent week (latest date only)
latest_date = df_weekly["date"].max()
df_latest_week = df_weekly[df_weekly["date"] == latest_date].copy()

In [33]:
# Preview result
print(f"✅ Pulled weekly data for: {latest_date.date()}")
df_latest_week

✅ Pulled weekly data for: 2025-08-03


,date,keyword,search_interest
260,2025-08-03,meditation,78
521,2025-08-03,mindfulness,22
782,2025-08-03,breathwork,4
1043,2025-08-03,guided meditation,5
1304,2025-08-03,yoga nidra,3


In [34]:
# 🔄 Purpose: Preview what the dataset will look like after appending new data

# Load the existing dataset
path_existing = "../data/streamlit/global_trend_summary.csv"
df_existing = pd.read_csv(path_existing, parse_dates=["date"])

# Check if this date is already present
latest_date = df_latest_week["date"].iloc[0]
if latest_date in df_existing["date"].unique():
    print(f"⚠️ Date {latest_date.date()} already exists in dataset. Skipping append.")
    df_preview = df_existing.copy()
else:
    # Append new row
    df_preview = pd.concat([df_existing, df_latest_week], ignore_index=True)
    print(f"✅ Appended new week: {latest_date.date()}")

✅ Appended new week: 2025-08-03


In [35]:
# Sort and preview
df_preview = df_preview.sort_values(["date", "keyword"]).reset_index(drop=True)
df_preview.tail(10)

,date,keyword,search_interest
1305,2025-07-27,breathwork,3
1306,2025-07-27,guided meditation,4
1307,2025-07-27,meditation,69
1308,2025-07-27,mindfulness,19
1309,2025-07-27,yoga nidra,3
1310,2025-08-03,breathwork,4
1311,2025-08-03,guided meditation,5
1312,2025-08-03,meditation,78
1313,2025-08-03,mindfulness,22
1314,2025-08-03,yoga nidra,3


In [37]:
# ✅ Save the updated global trend dataset using the correct DataFrame
df_preview.to_csv("../data/streamlit/global_trend_summary.csv", index=False)
print("✅ File successfully saved at: ../data/streamlit/global_trend_summary.csv")

✅ File successfully saved at: ../data/streamlit/global_trend_summary.csv


In [38]:
# 📅 Show the 10 most recent rows by date
df_preview.sort_values("date", ascending=False).head(10)

,date,keyword,search_interest
1314,2025-08-03,yoga nidra,3
1313,2025-08-03,mindfulness,22
1312,2025-08-03,meditation,78
1311,2025-08-03,guided meditation,5
1310,2025-08-03,breathwork,4
1309,2025-07-27,yoga nidra,3
1308,2025-07-27,mindfulness,19
1307,2025-07-27,meditation,69
1306,2025-07-27,guided meditation,4
1305,2025-07-27,breathwork,3
